In [27]:
import pandas as pd

df = pd.read_json('/home/stock-rex/Desktop/stock-reX/services/backend/data/shoe_metadata.json',lines=True)
display(df)

,_id,title,colorway,releaseDate,retailPrice,brand,model,tags,lowestAsk,numberOfAsks,...,annualHigh,annualLow,volatility,pricePremium,lastSale,changeValue,changePercentage,salesLast72Hours,sku,currency
0,{'$oid': '62ffd4071cdc4a9f390b6b29'},ASICS Gel-Kayano 14 JJJJound Silver Black,White/Black,None,NaN,Asics,ASICS Gel-Kayano 14,[],400,136,...,928,211,0.284326,0.000,253,-81,-0.244332,11,ee12884a-8a5d-4e8b-ad5f-f8b3c2c16e7d,GBP
1,{'$oid': '62ffd4121cdc4a9f390b762e'},adidas Yeezy Slide Onyx,Onyx/Onyx/Onyx,2022-03-07,60.0,adidas,adidas Yeezy Slide,[],83,3593,...,485,64,0.184443,0.700,86,-16,-0.163934,1953,ccc77727-c82d-48f8-8c61-9eb5b183a950,GBP
2,{'$oid': '62ffd42b1cdc4a9f390b8df0'},Nike Dunk Low Retro White Black Panda (2021) (GS),White/Black-White,2021-03-10,85.0,Nike,Nike Dunk Low Retro,[Low],117,1335,...,253,90,0.061512,1.141,154,11,0.076923,922,59f9f4de-a8b8-46fc-841f-42961d3567e7,GBP
3,{'$oid': '62ffd5391cdc4a9f390c8a9c'},Jordan 3 Retro Fire Red (2022),White/Fire Red/Cement Grey/Black,2022-09-10,210.0,Jordan,Jordan 3 Retro,[],216,908,...,431,156,0.099388,0.000,177,-2,-0.014085,516,2adf45ff-2287-4cd2-bb58-efb2b7e2f386,GBP
4,{'$oid': '62ffd5401cdc4a9f390c90a9'},Jordan 1 Low Shadow Toe,Light Smoke Grey/Black-White,2022-03-01,100.0,Jordan,Jordan 1 Low,[],117,1247,...,319,80,0.088879,0.550,131,-3,-0.025157,437,cd541468-6566-4fef-985b-12d293502f94,GBP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,{'$oid': '630014621cdc4a9f3946aec4'},Jordan 11 Retro Low IE Bred (2021),Black/White-True Red,2021-09-17,170.0,Jordan,Jordan 11 Retro Low IE,[],96,478,...,288,70,0.105426,-0.212,113,0,-0.007407,16,adc78b55-2a20-4cc4-b8a9-48aa052deed4,GBP
969,{'$oid': '6300147e1cdc4a9f3946c783'},Jordan 7 Retro Flint (2021),White/Flint Grey-Black-Varsity Purple,2021-05-08,190.0,Jordan,Jordan 7 Retro,[],149,878,...,330,59,0.136495,-0.158,135,16,0.126761,16,10ca8bfb-e40a-4ab0-b43a-4b9293de4213,GBP
970,{'$oid': '6300148e1cdc4a9f3946d634'},Nike Air Max 1 SP Concepts Far Out (Special Box),Wild Violet/Multi-Color-Sail,2022-03-26,230.0,Nike,Nike Air Max 1 SP,[],282,423,...,1050,171,0.248114,0.065,207,-3,-0.016064,14,289c01c5-c0c3-45cd-90c7-551b325b4e7c,GBP
971,{'$oid': '630014971cdc4a9f3946de1f'},Nike Dunk Low Vintage Navy (W),White/Midnight Navy,2022-03-04,100.0,Nike,Nike Dunk Low,[Low],279,248,...,560,161,0.183853,1.780,235,27,0.130202,15,e041ee35-350c-4668-8842-0328d2ffbb6c,GBP


In [51]:
import numpy as np
import re
import glob
import os

df = df.rename({"_id":"id"},axis=1)
df['id'] = [i for i in range(len(df))]

png_files = os.listdir('services/backend/data/shoe_pngs')
png_names = set([i.split('.png')[0] for i in png_files])

print(df['title'].isin(png_names))
final_df = df[df['title'].isin(png_names)]
final_df['png_path'] = final_df['title'] + '.png'
final_df.index = final_df['id']
token_df = df[['id','model']].copy()
final_df = final_df.drop(['tags'],axis=1)
del final_df['id']


0      False
1       True
2       True
3       True
4       True
       ...  
968     True
969     True
970    False
971     True
972     True
Name: title, Length: 973, dtype: bool


/tmp/ipykernel_24946/2154828320.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['png_path'] = final_df['title'] + '.png'


In [52]:
final_df.to_csv('clean_shoe_metadata.csv',header=True)

In [53]:
token_df['model'] = token_df['model'].str.split()
token_df = token_df.explode('model').reset_index(drop=True)
token_counts = token_df['model'].value_counts().reset_index()
token_counts.columns = ['token', 'total']

# Step 3: Merge the token counts back into the original dataframe
token_df = token_df.merge(token_counts, left_on='model', right_on='token', how='left')

# Drop the 'token' column, if needed
token_df = token_df.drop('token', axis=1)
token_df = token_df.rename({"id":"prod_id"},axis=1)
token_df.index.name = 'id'
# Display the result
print(token_df)

      prod_id       model  total
id                              
0           0       ASICS      1
1           0  Gel-Kayano      1
2           0          14      4
3           1      adidas    126
4           1       Yeezy     92
...       ...         ...    ...
3669      971         Low    278
3670      972      Jordan    383
3671      972           1    310
3672      972       Retro    224
3673      972        High    115

[3674 rows x 3 columns]


In [54]:
token_df.to_csv('shoe_tokens.csv',header=True)
